In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import tensorflow as tf
import statistics as stats
import random as rn
import os
import tensorflow.compat.v1 as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder 

from keras.models import Sequential
from keras.layers import Dense, Dropout, GaussianNoise
from keras.optimizers import Adam
from keras.regularizers import l1, l2
from keras import initializers as init

from tensorflow.python.keras import backend as K

from math import sqrt
from numpy.random import seed

from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer

Using TensorFlow backend.


In [2]:
# ensure repeatability
K.clear_session()
tf.reset_default_graph()

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(23)
rn.seed(23)
tf.random.set_random_seed(23)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [3]:
data = pd.read_csv("ML_data.csv", sep="|")
data2 = pd.read_csv("add_des.csv", sep="|", index_col=0)
pd.set_option("display.max_columns", None)

In [4]:
# Parameters
epochs=500
training_size=0.95
batch_size=50
activation='relu'
# learning_rate=1e-3
# adam_decay=1e-2
# dropout_rate=0.15
# noise=1.5e-2
# k_reg= 0.2 # kernal regularizer


In [5]:
############ DATA PRE-PROCESSING ############ 

# add in various functions to x
x = data.iloc[:, 10:15]
y = data.iloc[:, 6:10]
new_data = x.copy()
with np.errstate(divide='ignore', over='ignore'):
    new_data[x.columns + '^2']=(data[x.columns])**2
    new_data[x.columns + '^3']=(data[x.columns])**3
    new_data[x.columns + 'sqrt']=abs(data[x.columns])**(1/2)
    new_data[x.columns + 'cbrt']=abs(data[x.columns])**(1/3)
    new_data[x.columns + 'e^x']=np.exp(data[x.columns])
    new_data[x.columns + 'e^x-1']=np.expm1(data[x.columns])
    new_data[x.columns + '2^x']=np.exp2(data[x.columns])
    new_data[x.columns + 'lnx']=np.log(abs(data[x.columns]))
    new_data[x.columns + 'log10x']=np.log10(abs(data[x.columns]))
    new_data[x.columns + 'log2x']=np.log2(abs(data[x.columns]))
    new_data[x.columns + 'ln(1+x)']=np.log1p(abs(data[x.columns]))
new_data=new_data.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

# concatenate the newly derived functions from 5 "hot" columns, the rest
# of the descriptors, and the additional descriptors add_des csv (data2)
frames= [new_data, data.iloc[:, 15:52], data2]
func_data=pd.concat(frames, axis=1)

# separate categorical and continuous data 
categorical=pd.DataFrame()
continuous=pd.DataFrame()

for index in func_data.columns:
    if(func_data[index].dtypes == "int"):
        categorical[index]=func_data[index]
    elif(func_data[index].dtypes == "float"):
        continuous[index]=func_data[index]
    else:
        pass

# one hot encode categorical data
onehotencoder = OneHotEncoder()
categorical = onehotencoder.fit_transform(categorical).toarray()

# standardize continuous data
scaler = StandardScaler()
continuous = scaler.fit_transform(continuous)

# re-combine categorical and continuous data
x = np.concatenate((continuous, categorical), axis=1)

# extract y data and standardize (DFT predicted / output)
y = scaler.fit_transform(y)

# input and output dimension
in_dim = x.shape[1]
out_dim = y.shape[1]

In [6]:
# construct model

def construct_model(k_reg, dropout_rate, learning_rate, noise, adam_decay):
    
    # Create and add layers to model
    model = Sequential()
    #### Input layer
    model.add(Dense(1024,
                    input_dim=in_dim, 
                    activation=activation,
                    kernel_regularizer=l2(k_reg)
                    )
             )
    model.add(GaussianNoise(noise))
    model.add(Dropout(dropout_rate))
    model.add(Dense(512, activation=activation))
    model.add(Dense(512, activation=activation))
    model.add(Dense(out_dim)) #, activation='linear'))

    # configure optimizer & compile model
    opt = Adam(lr=learning_rate, decay=adam_decay)
    model.compile(loss="mse", optimizer=opt)

    #### summarize model
    print("Dropout Rate: " + str(dropout_rate))
    print("Kernal regularizer: " + str(k_reg))
    print("Noise: " + str(noise))
    print("Learning Rate: " + str(learning_rate))
    print("Adam Decay Rate: " + str(adam_decay))
    print(model.summary())
    
    return model

In [7]:
# Set parameter bounds to be optimized
learning_rate = Real(low=5e-4, high=1e-2, prior="log-uniform", name="learning_rate")
adam_decay = Real(low=0,high=0.1,name="adam_decay")
dropout_rate = Real(low=0, high=0.5, name="dropout_rate")
noise = Real(low=0, high=1e-1, name="noise")
k_reg = Real(low=0, high=0.5, name="k_reg")

dimensions = [learning_rate,
              adam_decay,
              dropout_rate,
              noise,
              k_reg
             ]

default_parameters = [1e-3, 1e-2, 0.15, 2e-2, 0.25]

In [8]:
@use_named_args(dimensions)
def train_model(k_reg, dropout_rate, learning_rate, noise, adam_decay, verbose=0):

    model = construct_model(k_reg, dropout_rate, learning_rate, noise, adam_decay)
    
    # split training and testing data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=(1-training_size), random_state=23)
    
    # train model
    prediction = model.fit(x_train, y_train,
                          validation_data=(x_test, y_test),
                          epochs=epochs,
                          batch_size=batch_size,
                          verbose=verbose)
    
    # Finding mean RMSE of testing data
    pred_test = model.predict(x_test)
    RMSE_test = (stats.mean([sqrt(mean_squared_error(y_test[:,0], 
                                                     pred_test[:,0])),
                             sqrt(mean_squared_error(y_test[:,1], 
                                                    pred_test[:,1])),  
                             sqrt(mean_squared_error(y_test[:,2], 
                                                    pred_test[:,2])),
                             sqrt(mean_squared_error(y_test[:,3], 
                                                    pred_test[:,3]))
                            ])
                )
    
    # clear session & reset model graphs
    K.clear_session()
    tf.reset_default_graph()

    return RMSE_test

In [9]:
# Run Bayesian Optimization and return its results
bayopt_results = gp_minimize(func=train_model, 
                             dimensions=dimensions, 
                             n_random_starts=150,
                             n_calls=340, 
                             n_jobs=-1,
                             kappa=5,
                             x0=default_parameters,
                             random_state=23,
                             verbose=True
                            )

Iteration No: 1 started. Evaluating function at provided point.
Dropout Rate: 0.15
Kernal regularizer: 0.25
Noise: 0.02
Learning Rate: 0.001
Adam Decay Rate: 0.01
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              331776    
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_4 (Dense)        

Iteration No: 6 ended. Evaluation done at random point.
Time taken: 186.1623
Function value obtained: 0.3657
Current minimum: 0.2919
Iteration No: 7 started. Evaluating function at random point.
Dropout Rate: 0.21636480044493483
Kernal regularizer: 0.2353031070111375
Noise: 0.08760779531940954
Learning Rate: 0.0009440347289667215
Adam Decay Rate: 0.08747145186848154
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              331776    
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_______________

Iteration No: 12 ended. Evaluation done at random point.
Time taken: 176.9491
Function value obtained: 0.3209
Current minimum: 0.2919
Iteration No: 13 started. Evaluating function at random point.
Dropout Rate: 0.4854924274515475
Kernal regularizer: 0.39790667562911597
Noise: 0.04125502797309408
Learning Rate: 0.005038838065174854
Adam Decay Rate: 0.07786513483448015
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              331776    
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
______________

Iteration No: 18 ended. Evaluation done at random point.
Time taken: 176.0439
Function value obtained: 0.3530
Current minimum: 0.2919
Iteration No: 19 started. Evaluating function at random point.
Dropout Rate: 0.3908999442729749
Kernal regularizer: 0.04998324348691614
Noise: 0.031936008182206196
Learning Rate: 0.0045384733127000025
Adam Decay Rate: 0.029202926253234424
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              331776    
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
___________

Iteration No: 24 ended. Evaluation done at random point.
Time taken: 196.3993
Function value obtained: 0.3284
Current minimum: 0.2919
Iteration No: 25 started. Evaluating function at random point.
Dropout Rate: 0.2479866577623307
Kernal regularizer: 0.028686823842636773
Noise: 0.040653281869006457
Learning Rate: 0.0005383607503583506
Adam Decay Rate: 0.0028030089252449215
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              331776    
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________

...

Iteration No: 312 ended. Search finished for the next optimal point.
Time taken: 211.7646
Function value obtained: 0.3151
Current minimum: 0.2791
Iteration No: 313 started. Searching for the next optimal point.
Dropout Rate: 0.02595398646259329
Kernal regularizer: 0.1768557738012421
Noise: 0.002392475430967945
Learning Rate: 0.003066896767752327
Adam Decay Rate: 0.009327146062336035
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              331776    
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800   

Iteration No: 318 ended. Search finished for the next optimal point.
Time taken: 209.0776
Function value obtained: 0.3080
Current minimum: 0.2791
Iteration No: 319 started. Searching for the next optimal point.
Dropout Rate: 0.5
Kernal regularizer: 0.12843290727762793
Noise: 0.0
Learning Rate: 0.0005
Adam Decay Rate: 0.0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              331776    
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_____________________________________________________________

Iteration No: 324 ended. Search finished for the next optimal point.
Time taken: 217.2585
Function value obtained: 0.3007
Current minimum: 0.2791
Iteration No: 325 started. Searching for the next optimal point.
Dropout Rate: 0.261176994439818
Kernal regularizer: 0.4885973708269253
Noise: 0.0013495155436983436
Learning Rate: 0.009067157143717973
Adam Decay Rate: 0.09099676757208712
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              331776    
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    


Iteration No: 330 ended. Search finished for the next optimal point.
Time taken: 217.7100
Function value obtained: 0.3321
Current minimum: 0.2734
Iteration No: 331 started. Searching for the next optimal point.
Dropout Rate: 0.23731097629779063
Kernal regularizer: 0.49657149065527956
Noise: 0.0008393043808587854
Learning Rate: 0.0007414782991823368
Adam Decay Rate: 0.0133397677900603
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              331776    
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800  

Iteration No: 336 ended. Search finished for the next optimal point.
Time taken: 217.7680
Function value obtained: 0.3127
Current minimum: 0.2734
Iteration No: 337 started. Searching for the next optimal point.
Dropout Rate: 0.034390354876785885
Kernal regularizer: 0.1482815663702042
Noise: 3.868465252743204e-06
Learning Rate: 0.0005022180667334429
Adam Decay Rate: 0.05070753085692693
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              331776    
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800 

In [30]:
column_names = ["learning_rate",
                "adam_decay",
                "dropout_rate",
                "noise",
                "k_reg",
               ]

iterations = pd.DataFrame(data=bayopt_results.x_iters, 
                          index=bayopt_results.func_vals, columns=column_names)
print("Best Parameters: "  + str(bayopt_results.x))
# iterations.to_excel(r'2nd_Large_BayOpt.xlsx')
iterations

Best Parameters: [0.003746732751708163, 0.006709578082719618, 0.005789320226934709, 8.381377063769025e-05, 0.24636233956533735]


,learning_rate,adam_decay,dropout_rate,noise,k_reg
0.291948,0.001000,0.010000,0.150000,0.020000,0.250000
0.346963,0.003711,0.018499,0.390840,0.057391,0.220957
0.328404,0.000868,0.009982,0.109826,0.083334,0.213383
0.361916,0.004794,0.099687,0.000042,0.097595,0.209561
0.341530,0.003852,0.075263,0.289429,0.039494,0.239787
...,...,...,...,...,...
0.312719,0.009676,0.038978,0.053527,0.002137,0.158409
0.327433,0.000502,0.050708,0.034390,0.000004,0.148282
0.326388,0.008131,0.064390,0.006317,0.000660,0.222469
0.367473,0.000532,0.096934,0.009735,0.000714,0.161232
